In [1]:
from shared.preface import *
import shared.functions as fct


#1. Read in all velocities for neutrinos.
nus = 40000
sim = 'L012N376'
mass_gauge = 11.5  # in log10 Msun
mass_range = 0.1
halo_num = 1
halo_j = 0
batchname = f'1e+{mass_gauge}_pm{mass_range}Msun'
hname = f'{batchname}_halo{halo_j}'
fname = f'{nus}nus_{hname}'
u_all = fct.load_sim_data(sim, fname, 'velocities')

#2. Convert to momenta for one test mass and get "Fermi-Dirac weights".
m_nu = 0.3*eV
p, _ = fct.u_to_p_eV(u_all, m_nu)
p0, p1 = p[:,0], p[:,-1]
FD_weights = p0**2 * fct.Fermi_Dirac(p1)

#3. Reshape into angular coord. batches, i.e. (phi, theta) batches. Each 
#   coord. pair has an amount of Vs neutrinos.
p0_angular = np.reshape(p0, (PHIs, THETAs, Vs))
p1_angular = np.reshape(p1, (PHIs, THETAs, Vs))

#4. The number density at coords. (phi, theta) in the sky is the integral over 
#    all momenta with that coord. pair.
p0_pairs = np.reshape(p0_angular, (PHIs*THETAs, Vs))
p1_pairs = np.reshape(p1_angular, (PHIs*THETAs, Vs))
print(p0_pairs.shape)

ODs = np.zeros(len(p0_pairs))
for i, (p0s_pair, p1s_pair) in enumerate(zip(p0_pairs, p1_pairs)):
    ODs[i] = fct.number_density(p0s_pair, p1s_pair)/N0

print(ODs)

(100, 400)
[2.49699567 2.07190943 2.27721245 2.60322497 3.86514634 2.85430729
 2.19774128 2.21402954 1.80144396 1.75184969 2.3878548  2.03771836
 2.30343053 1.94733512 5.92232693 1.88393868 2.10094573 2.00974733
 1.75305955 1.56280765 2.18334485 2.44156511 1.77517226 1.83354619
 1.97683483 2.33822079 2.20083065 1.62064029 2.07338077 2.4126155
 2.52413639 2.18932103 2.23559957 1.93297731 1.53517588 2.06631563
 2.11723015 1.63769001 2.08511959 1.9782056  2.4898516  2.77200159
 2.51759467 1.90604183 2.66850185 2.54519399 1.90569559 2.6879993
 2.22562789 1.36836932 2.49301288 2.24840563 1.78305752 1.96604554
 1.68373775 1.67613839 1.78676872 2.21019658 2.33671562 2.13229812
 2.54468574 2.27965415 1.68217896 1.89239121 2.1797105  2.11611396
 2.12823486 2.06578167 6.73455311 1.46943912 2.60610154 2.29879977
 2.0616435  2.39013107 2.28675363 2.29895737 2.60919757 3.0613124
 3.71255117 1.62128839 2.52402718 2.44803857 2.29744788 2.4788027
 2.54328721 2.70570238 2.92876402 3.03059456 2.60435446

In [24]:
eps = 0.01  # shift in theta, so poles are not included
phis = np.linspace(0., 2.*Pi, PHIs)
thetas = np.linspace(0.+eps, Pi-eps, THETAs)

Nside = 2**1              # Specify nside parameter
Npix = 12 * Nside**2      # Number of pixels
print(Npix)
pix_sr = (4*np.pi)/Npix   # Pixel size  [sr]

# Pixel index for each coord.
pix_IDs = np.array(
    [hp.pixelfunc.ang2pix(Nside, phi=p, theta=t) for p in phis for t in thetas]
)

sort_IDs = pix_IDs.argsort()
# print(pix_IDs[sort_IDs])

ODs_sort = ODs[sort_IDs]



# Create random poisson sky-map
# poisson_map = np.random.poisson(100,Npix)

# hp.mollview(poisson_map, title='', unit='')
# hp.graticule()

48
